In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

fig_DIR = "../figs/players/"

export_DIR = "../../repos/csv/"

In [ ]:
def get_df(stats):
    df = stats.get_data_frames()[0]
    # df = df[df["MIN"] >= 15]
    df = df.rename(columns={"PLAYER_NAME":"Player","TEAM_ABBREVIATION":"Team"})
    cols = df.columns
    df.columns = cols.str.replace("RATING", "RTG")
    df.columns = map(str.lower,df.columns)
    return df

In [ ]:
stats = leaguedashplayerstats.LeagueDashPlayerStats(
    measure_type_detailed_defense="Base", per_mode_detailed="Totals",
    league_id_nullable="20",season="2024-25",season_type_all_star="Showcase"
)
df1 = get_df(stats)

In [ ]:
stats = leaguedashplayerstats.LeagueDashPlayerStats(
    measure_type_detailed_defense="Base", per_mode_detailed="Totals",
    league_id_nullable="20",season="2024-25",season_type_all_star="Regular Season"
)
df2 = get_df(stats)

In [ ]:
df11 = df1.iloc[:,:32]
df11["segment"] = "show"
df21 = df2.iloc[:,:32]
df21["segment"] = "regular"

In [ ]:
df3 = pd.concat([df21,df11])

In [ ]:
df3.columns

In [ ]:
df3g = df3.groupby(['player_id', 'player', 'nickname', 'team_id','age',])
keys = list(df3g.groups)


In [ ]:
dfa = []
for key in tqdm(keys):
    df4 = df3g.get_group(key).reset_index(drop=True)
    df4.index +=1
    df4.loc[0] = df4.loc[1]
    df4.iloc[-1,6:-1] = df4.iloc[:2,6:-1].sum()
    df4.iloc[-1,-1] = "comb"
    df4["w_pct"] = round(df4["w"]/(df4["gp"]),3)
    df4["fg_pct"] = round(df4["fgm"]/df4["fga"],3)
    df4["fg3_pct"] = round(df4["fg3m"]/df4["fg3a"],3)
    df4["ft_pct"] = round(df4["ftm"]/df4["fta"],3)
    dfa.append(df4.tail(1))
df5 = pd.concat(dfa)

In [ ]:
df6 = df5.reset_index(drop=True)
df6.iloc[:,10:-1]  = round(df6.iloc[:,10:-1].div(df6.gp,axis=0),2)
df6["w_pct"] = round(df6["w"]/(df6["gp"]),3)
df6["fg_pct"] = round(df6["fgm"]/df6["fga"],3)
df6["fg3_pct"] = round(df6["fg3m"]/df6["fg3a"],3)
df6["ft_pct"] = round(df6["ftm"]/df6["fta"],3)
df6 = df6.drop(columns=["segment"])
df6 = df6.sort_values("pts",ascending=False).reset_index(drop=True)
df6 = df6.reset_index(drop=True)
df6.index +=1

In [ ]:
df6.to_csv(export_DIR + "GLeague_Box_P_Base_Cum_2024.csv")